# 1-

In [ ]:
import os
import base64
import tempfile
from openai import AzureOpenAI
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, Form, File
from fastapi.responses import JSONResponse
from pathlib import Path
from fastapi.middleware.cors import CORSMiddleware

# 2-

In [ ]:
RE_OPENAI_API_KEY =                # Chave API
RE_OPENAI_ENDPOINT =               # URL até antes do /openai
RE_OPENAI_API_VERSION =            # Versão da API
RE_OPENAI_DEPLOYMENT_NAME =        # Nome da implantação

# 3-

In [ ]:
env_path = Path(__file__).resolve(strict=True).parent / ".env"
load_dotenv(dotenv_path=env_path)

# Carregar as variáveis de ambiente do arquivo . env
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Configuração do FastAPI
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],     # Permitir todas as origens
    allow_credentials=True,
    allow_methods=["*"],     # Permitir todos os métodos
    allow_headers=["*"],     # Permitir todos os cabeçalhos
)

# Configuração do cliente OpenAI
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME
)

def criar_prompt_modelo_ameacas(tipo_aplicacao,
                                autenticacao,
                                acesso_internet,
                                dados_sensiveis,
                                descricao_aplicacao):
    prompt = f"""Aja como um especialista em cibersegurança com mais de 20 anos de experiência utilizando a metodologia de modelagem de ameaças STRIDE para produzir modelos de ameaças abrangentes para uma ampla gama de aplicações. Sua tarefa é analisar o resumo do código, o conteúdo do README e a descrição da aplicação fornecidas para produzir uma lista de ameaças especificas para essa aplicação.

    Preste atenção na descrição da aplicação e nos detalhes técnicos fornecidos.

    Para cada uma das categorias do STRIDE (Falsificação de Identidade - Spoofing, Violação de integridade - Tampering, Repúdio - repudiation, Divulgação de Informações - Information Disclosure, Negação de serviço - Denial of service, Elevação de Privilégio - Elevation of Privilege), liste múltiplas (3 ou 4) ameaças reais se aplicável. Cada cenário de ameaça deve apresentar uma situação plausivel em que a ameaça poderia ocorrer no contexto da aplicação.

    A lista de ameaças deve ser apresentada em formato tabela, com as seguintes colunas: ao fornecer o modelo de ameaças, utilize uma resposta formato JSON com as chaves "threat_model" e "improvement_suggestions". Em "threat_model", inclua um array de objeto com as chaves "Threat Type" (Tipo de Ameaça), "Scenario" (cenário), e "Potential impact" (Impacto potencial).

    Ao fornecer o modelo de ameaças, utilize um resposta formatada em JSON com as chaves "threat_model" e "improvement-suggestions". Em "threat_model", inclua um array de objetos com as chaves "Threat Type" (Tipo de ameaças), "Scenario" (Cenário), e "Potencial impact" (Impacto potencial).

    Em "improvement_suggestions", inclua um array de strings que sugerem quais informações adicionais poderiam ser fornecidas para tornar o modelo de ameaças mais completo e preciso na próxima iteração. Foque em identificar lacunas na descrição da aplicação que, se preenchidas, permitiriam uma análise mais detalhada e precisa, como por exemplo:
    - Detalhes arquiteturais ausentes que ajudariam a identificar ameaças mais especificas
    - Fluxos de autenticação pouco claros que precisam de mais detalhes
    - Descrição incompleta dos fluxos de dados
    - informação técnica de stack não informadas
    - Fronteiras ou zonas de confiança do sistema não especificadas
    - descrição incompleta do tratamento de dados sensíveis

    Não forneça recomendações de segurança genéricas - foque apenas no que ajudaria a criar um modelos de ameaças mais eficiente.

    TIPO DE APLICAÇÃO: {tipo_aplicacao}
    MÉTODOS DE AUTENTICAÇÃO: {autenticacao}
    ACESSO A INTERNET: {acesso_internet}
    DADOS SENSIVEIS: {dados_sensiveis}
    RESUMO DE CÓDIGO, CONTEÚDO DO README E DESCRIÇÃO DA APLICAÇÃO: {descricao_aplicacao}

    Exemplode formato esperado em JSON:

        {{
          "threat_model": [
            {{
              "Threat Type": "Spoofing",
              "Scenario": "Cenário de exemplo 1",
              "Potential impact": "Impacto potencial de exemplo 1"
            }},
            {{
              "Threat Type": "Tampering",
              "Scenario": "Cenário de exemplo 2",
              "Potential impact": "Impacto potencial de exemplo 2"
            }}
            // ... mais ameaças
          ],
          "improvement_suggestions": [
            "Sugestão de melhoria 1",
            "Sugestão de melhoria 2",
            // ... mais sugestões
          ]
        }}"""

    return prompt

# 4-

In [ ]:
@app.post("/analisar_ameacas")
async def analisar_ameacas(
    imagem:UploadFile = File(...),
    tipo_aplicacao: str = Form(...),
    autenticacao: str = Form(...),
    acesso_internet: str = Form(...),
    dados_sensiveis: str = Form(...),
    descricao_aplicacao: str = Form(...)
):
    try:
        # Criar prompt para o modelo de ameaças
        prompt = criar_prompt_modelo_ameacas(tipo_aplicacao,
                                             autenticacao,
                                             acesso_internet,
                                             dados_sensiveis,
                                             descricao_aplicacao)

        # Salvar a imagem temporariamente
        with tempfile.NamedTemporaryFile(delete=False, suffix=Path(imagem.filename).suffix) as temp_file:
            content = await imagem.read()
            temp_file.write(await imagem.read())
            temp_file_path = temp_file.name

        # Convert imagem para base64
        with open(temp_file_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

        # Adicionar imagem codificada ao prompt
        chat_prompt = [
            {"role": "system", "content": "Você é uma IA especialista em cibersegurança"},
            {"role": "user", "content": {"type": "text", "text":prompt}},
            {"role": "user", "content": {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_string}"}}
        ]


        # Chamar o modelo OpenAI
        response = client.chat(
            messages= chat_prompt,
            temperature=0.7,
            max_tokens=1500,
            top_p=0.95
        )
        os.remove(temp_file_path) # Remover o arquivo temporário após o uso

        # Retornar a resposta do modelo
        return JSONResponse(content=response.to_dict(), status_code=200)

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)